In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import numpy as np

In [7]:
# word index
word_index=imdb.get_word_index()
reversed_index={value:key for key,value in word_index.items()}

In [8]:
model=load_model('simple_rnn_imdb.h5')

In [9]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [11]:
model.weights

[<Variable path=sequential_3/embedding_3/embeddings, shape=(10000, 128), dtype=float32, value=[[-0.98528373 -0.06685684 -0.0100947  ... -0.80757415 -1.2233129
    0.29445958]
  [-0.03813602  0.05417977 -0.08060603 ... -0.05040999 -0.05303597
    0.06529865]
  [-0.1635175  -0.0780634   0.1092672  ... -0.07848422 -0.10362577
   -0.09270202]
  ...
  [ 0.0138399  -0.053385    0.00557308 ...  0.02945296 -0.03162849
    0.00639671]
  [-0.07100738  0.01807276 -0.01607534 ... -0.02324806 -0.07034172
   -0.00799871]
  [-0.02273667 -0.03365465  0.00712486 ...  0.03764286  0.01279387
   -0.00123092]]>,
 <Variable path=sequential_3/simple_rnn_3/simple_rnn_cell/kernel, shape=(128, 128), dtype=float32, value=[[ 0.0619304  -0.02434951 -0.0664655  ...  0.1402017   0.04860546
    0.16145279]
  [-0.1007194   0.06279299 -0.02962367 ...  0.09160454  0.08738688
    0.0396415 ]
  [-0.02033703 -0.1019429  -0.11423683 ...  0.08790569 -0.02442393
   -0.06216531]
  ...
  [ 0.11483695  0.117635    0.11017592 ...

In [13]:
# function to decode reviews
def decode_reviews(encodes_reviews):
    return ' '.join([reversed_index.get(i-3,'?') for i in encodes_reviews])

# function to preprocess user input
def preprocess_text(text):
    words=text.lower().split()
    encoded_review=[word_index.get(word,2)+3 for word in words]
    padded_review=pad_sequences([encoded_review],maxlen=500)
    return padded_review

#### Why Add 3?
Adding a constant offset like 3 might be done for several reasons:

To reserve certain numbers for special tokens, such as:
0: For padding.
1: For start of a sequence.
2: For unknown words.
Adding 3 ensures the actual word indices don't conflict with these reserved tokens.


In [51]:
# prediction function
def predict_sentiment(review):
    preprocessed=preprocess_text(review)
    prediction=model.predict(preprocessed)

    sentiment='positive' if prediction[0][0] > 0.4 else 'negative'
    return sentiment,prediction

In [52]:
# example review for prediction
ex_review="very good"
sentiment,score=predict_sentiment(ex_review)
print(f'Review :{ex_review}')
print(f'Sentiment : {sentiment}\nScore : {score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Review :very good
Sentiment : positive
Score : [[0.4774523]]
